In [1]:
import json
import os
import pandas as pd
import numpy as np
import itertools
import joblib

In [2]:
date = '20190328'
folder = '..\\..\\..\\sportfolio\\mlb\\json'

In [3]:
files = os.listdir(folder)
files = [f for f in files if 'json' in f]

In [4]:
map_dict = {
    'LAA' : 'ANA', 
    'CWS' : 'CHA',
    'CHC' : 'CHN',
    'KC' : 'KCA',
    'LAD' : 'LAN',
    'NYY' : 'NYA',
    'NYM' : 'NYN',
    'SD' : 'SDN',
    'SF' : 'SFN',
    'STL' : 'SLN',
    'TB' : 'TBA',
    'WSH' : 'WAS'}
teams_to_change = list(map_dict.keys())

In [5]:
def get_teams_pitchers(game, site):
    away_team = game['details']['away']['team']
    if away_team in teams_to_change:
        away_team = map_dict[away_team]
    home_team = game['details']['home']['team']
    if home_team in teams_to_change:
        home_team = map_dict[home_team]
    away_pitcher = game['details']['away']['pitcher']
    home_pitcher = game['details']['home']['pitcher']
    return away_team, away_pitcher, home_team, home_pitcher

def get_spreads_totals_odds(game, site):
    if len(game['spreads']) - 1 < site:
        return 0, 0, 0, 0, 0, 0
    spread = float(game['spreads'][site]['home_value'])
    odds_home = int(game['spreads'][site]['home_odds'])
    odds_away = int(game['spreads'][site]['away_odds'])
    total = game['totals'][site]['home_value']
    over_odds = int(game['totals'][site]['away_odds'])
    under_odds = int(game['totals'][site]['home_odds'])
    return spread, odds_away, odds_home, total, over_odds, under_odds

In [6]:
site = 1
gameids = []
away_teams = {}
home_teams = {}
away_pitchers = {}
home_pitchers = {}
spreads = {}
away_spread_odds = {}
home_spread_odds = {}
totals = {}
over_odds = {}
under_odds = {}
for file_ in files:
    path = folder+'\\'+file_
    with open(path) as f:
        data = json.load(f)
    date = file_.split('.')[0]
    away_teams[date] = {}
    home_teams[date] = {}
    away_pitchers[date] = {}
    home_pitchers[date] = {}
    spreads[date] = {}
    away_spread_odds[date] = {}
    home_spread_odds[date] = {}
    totals[date] = {}
    over_odds[date] = {}
    under_odds[date] = {}
    print(date)
    home_team = []
    for i, game in enumerate(data[date]):
        j=0
        [away_teams[date][i], away_pitchers[date][i], 
        home_teams[date][i], home_pitchers[date][i]] = get_teams_pitchers(game, site)
        if home_teams[date][i] in home_team:
            j+=1
        else:
            home_team += [home_teams[date][i]]
        gameid = home_teams[date][i]+date+str(j)
        [spreads[date][gameid], away_spread_odds[date][gameid], 
        home_spread_odds[date][gameid], totals[date][gameid], 
        over_odds[date][gameid], under_odds[date][gameid]] = get_spreads_totals_odds(game, site)
        if totals[date][gameid]!=0:
            gameids += [gameid]
        else:
            print('hi')

20190328
20190329
20190330
20190331
20190401
20190402
20190403
20190404
20190405
20190406
20190407
20190408
20190409
20190410
20190411
20190412
hi
20190413
20190414
hi
20190415
20190416
20190417
20190418
20190419
20190420
20190421
20190422
20190423
20190424
20190425
20190426
20190427
20190428
20190429
20190430
20190501
20190502
20190503
20190504
20190505
20190506
20190507
20190508
20190509
20190510
20190511
20190512
20190513
20190514
20190515
20190516
20190517
20190518
20190519
20190520
20190521
20190522
20190523
20190524
20190525
20190526
20190527
20190528
20190529
20190530
20190531
20190601
20190602
20190603
20190604
20190605
20190606
20190607
20190608
20190609
20190610
20190611
20190612
20190613
20190614
20190615
20190616
20190617
20190618
20190619
20190620
20190621
20190622
20190623
20190624
20190625
20190626
20190627
20190628
20190629
20190630
20190701
20190702
20190703
20190704
20190705
20190706
20190707
20190708
hi
hi
hi
hi
hi
hi
hi
20190709
20190710
20190711
20190712
20190713
2

In [7]:
simulations = joblib.load('../data/processed2/2019_simulations_v2_3.joblib')
simulations1 = joblib.load('../data/processed2/2019_simulations_v4_3.joblib')
df_results = pd.read_csv('../data/processed2/2019_f5_scores.csv', index_col=0)

In [8]:
games = list(simulations.keys())

In [9]:
games = [game for game in games if game[-1] == '0']

In [10]:
a = set(games)
b = set(df_results.index)

In [11]:
do_not_include = list(a.symmetric_difference(b))

In [12]:
df_results = df_results.loc[[game for game in games if game not in do_not_include]]

In [13]:
gameids = [game for game in gameids if '20190708' not in game]
gameids = [game for game in gameids if '20190709' not in game]

In [14]:
games = set(list(df_results.index))
games_bets = set(gameids[233:])

In [15]:
all_games = games - (games - games_bets)

In [16]:
dates = np.unique([game[3:11] for game in all_games])

In [20]:
for date in dates:
    games = [game for game in all_games if date in game]
    for game in games:
        spread_median = np.median(simulations1[game][:, 1] - simulations1[game][:, 0])
        total_median = np.median(simulations1[game][:, 1] + simulations1[game][:, 0])
        predicted_spread_diff = spread_median - spreads[date][game]
        predicted_total_diff = total_median - totals[date][game]
        if predicted_spread_diff > 0:
            odds_spread = home_spread_odds[date][game]
        elif predicted_spread_diff < 0:
            odds_spread = away_spread_odds[date][game]
        else:
            print('weird')
        actual_spread = 

.  0.]
 [ 4.  3.]
 [ 2.  1.]
 [ 6.  0.]
 [ 4.  1.]
 [14.  1.]
 [ 7.  0.]
 [ 8.  8.]
 [ 3.  5.]
 [ 5.  1.]
 [ 8.  2.]
 [ 5.  7.]
 [ 0.  2.]
 [ 2.  0.]
 [ 2.  2.]
 [ 1.  5.]
 [ 1.  4.]
 [ 0.  3.]
 [ 6.  2.]]
[[ 5.  1.]
 [ 2.  2.]
 [ 2.  0.]
 [ 9.  4.]
 [ 0.  3.]
 [ 0.  1.]
 [ 4.  0.]
 [ 2.  5.]
 [ 4.  3.]
 [ 1.  1.]
 [ 6.  3.]
 [ 4.  5.]
 [ 3.  1.]
 [ 0.  3.]
 [ 5.  0.]
 [ 1.  1.]
 [ 8.  1.]
 [ 4.  0.]
 [ 4.  0.]
 [ 5.  3.]
 [ 3.  4.]
 [ 2.  2.]
 [ 7.  0.]
 [ 7.  1.]
 [ 0.  1.]
 [ 7.  5.]
 [ 2.  1.]
 [ 0.  2.]
 [ 3.  1.]
 [ 1.  3.]
 [ 0.  0.]
 [ 3.  1.]
 [ 3.  1.]
 [ 3.  0.]
 [ 2.  0.]
 [ 0.  1.]
 [ 1.  3.]
 [ 2.  0.]
 [ 1.  1.]
 [ 2.  0.]
 [ 7.  5.]
 [ 4.  1.]
 [ 3.  7.]
 [ 0.  0.]
 [ 6.  4.]
 [11.  0.]
 [ 0.  4.]
 [ 0.  2.]
 [ 1.  4.]
 [ 2.  3.]
 [ 1.  1.]
 [ 9.  2.]
 [ 1.  1.]
 [ 7.  0.]
 [ 6.  3.]
 [ 1.  6.]
 [ 3.  7.]
 [ 6.  6.]
 [ 2.  4.]
 [ 3.  0.]
 [ 0.  3.]]
[[ 5.  6.]
 [ 1.  1.]
 [ 7. 14.]
 [ 1.  6.]
 [ 4.  1.]
 [10.  0.]
 [ 0.  3.]
 [11. 11.]
 [ 3.  1.]
 [ 3.  0.]
 [ 5.  4.]
 